In [17]:
from sklearn.linear_model import LinearRegression  # 선형회귀
from sklearn.preprocessing import PolynomialFeatures # 다항특성을 만들어주는 라이브러리
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False  # 마이너스 표시 해결
# 한글설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # windows 사용자
# matplotlib.rcParams['font.family'] = 'AppleGothic Gothic' # Mac사용자
matplotlib.rcParams['font.size'] = '10' # 글자크기

alcohol sugar pH: 수소이온농도 - 화이트, 레드와인을 분류하는 모델 구현

In [18]:
wine = pd.read_csv('https://bit.ly/wine-date')
wine

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

In [ ]:
# 데이터 확인

# nan,0, 타입, 데이터 간격
# str에 대한 데이터 분리
# wine.describe()
# wine.info()
# 결과값의 파악 : class 
# hist그래프 출력
# 각각 피쳐간 상관관계 corr, heatmap
# - 피쳐간 관계 : 비슷한 피쳐는 삭제
# 각각 피쳐를 출력 subplots 각각 데이터 분포 파악 
# ( 0 데이터가 어느정도 있는지, 데이터 분포가 어떻게 되는지 )

# 결과값 파악

In [ ]:
wine.info()

NameError: name 'wine' is not defined

In [ ]:
wine.describe()

NameError: name 'wine' is not defined

In [ ]:
# 결과 0,1
wine['class'].unique()

NameError: name 'wine' is not defined

In [ ]:
wine['class'].value_counts()

NameError: name 'wine' is not defined

In [ ]:
labels = ['화이트와인','레드와인']
plt.pie(wine['class'].value_counts(),
        labels = labels, autopct='%0.1f%%',startangle=90, counterclock=False)

plt.legend(loc=(1,0.5))
plt.show()

NameError: name 'wine' is not defined

In [ ]:
import seaborn as sns

In [ ]:
corr = wine.corr(method='pearson')
sns.heatmap(corr,annot=True)

In [ ]:
wine.columns

In [ ]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
# 0.25
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target = train_test_split(
    data,target, random_state=42
)

In [ ]:
# 표준화 작업
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
# 데이터 실습
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_scaled,train_target)

print(lr.score(train_scaled,train_target))
print(lr.score(test_scaled,test_target))

In [ ]:

### 경사하강법 머신러닝 훈련
from sklearn.linear_model import SGDClassifier

In [ ]:
sc = SGDClassifier(loss='log_loss' ,random_state=42)

train_score = []
test_score = []
classes = np.unique(train_target)

In [ ]:
for _ in range(0,300):
    sc.partial_fit(train_scaled,train_target,classes = classes)
    train_score.append(sc.score(train_scaled,train_target))
    test_score.append(sc.score(test_scaled,test_target))

In [ ]:
plt.plot(train_score)
plt.plot(test_score)

In [ ]:
# 피쳐 너무 적고, 정답률 높지 않아서... 다른 모델을 대입해서 사용

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_scaled,train_target)
print(dt.score(train_scaled,train_target))
print(dt.score(test_scaled,test_target))

# 순수노드 나올때까지 훈련
# 과대적합 - 85% 대면 높은 정답률을 가짐

In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(10,7))
plot_tree(dt)
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
# max_depth : 가지의 깊이 조정
# filled : 색상조정, 찐한 파란색-양성클래스,1 주황색-음성클래스,0
plot_tree(dt, max_depth=2,filled=True,feature_names=['alcohol','sugar','pH'])
plt.show()

In [ ]:
# 가지치기를 해서 과대 적합 문제를 해결
# 3,4,5,6...10 가장 효율이 좋은 모델을 적용
dt = DecisionTreeClassifier(max_depth=3,random_state=42)
dt.fit(train_scaled,train_target)
print(dt.score(train_scaled,train_target))
print(dt.score(test_scaled,test_target))

In [ ]:

wine.columns

In [ ]:

plt.figure(figsize=(20,15))
plot_tree(dt,filled=True,feature_names=wine.columns)
plt.show()

In [ ]:
# 가지치기를 해서 과대 적합 문제를 해결
# 3,4,5,6...10 가장 효율이 좋은 모델을 적용
dt = DecisionTreeClassifier(max_depth=3,random_state=42)
dt.fit(train_scaled,train_target)
print(dt.score(train_scaled,train_target))
print(dt.score(test_scaled,test_target))

In [ ]:
# min_impurity_decrease 최소 불순도 조정해서 속도개선, 효율도 올릴수 있도록 모델을 조정
# 부모의 불순도 - 자식의 불순도 크기 큰 쪽을 선택, 차이가 0.005 도 안나면 그 노드에서 결정
# 과대적합 해결 - max_depth, min_impurity_decrease
dt = DecisionTreeClassifier(min_impurity_decrease=0.001,random_state=42)
dt.fit(train_scaled,train_target)
print(dt.score(train_scaled,train_target))
print(dt.score(test_scaled,test_target))